In [2]:
import re
import spacy
import csv
import pandas as pd
from spacy.lang.en.examples import sentences 
from geopy.distance import great_circle
from geopy.exc import GeocoderTimedOut, GeocoderServiceError

In [26]:
from geopy.geocoders import GoogleV3
geolocator = GoogleV3(api_key="...")

In [4]:
nlp = spacy.load("en_core_web_sm")

In [5]:
nlp.pipe_names

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']

In [6]:
news_df = pd.read_csv('/Users/gayathri/Library/CloudStorage/Box-Box/Local News Data/data/local_news_data.csv')
news_df.columns


Index(['crawl_date', 'domain', 'url', 'mime_type_web_server', 'mime_type_tika',
       'language', 'content'],
      dtype='object')

In [7]:
print(nlp.pipe_names)

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']


In [8]:
news_df.head(10)

,crawl_date,domain,url,mime_type_web_server,mime_type_tika,language,content
0,20190930151555,themontynews.org,https://www.themontynews.org/,text/html,text/html,en,"""Home | The Montgomery News No challenge befor..."
1,20190930151853,themontynews.org,https://www.themontynews.org/,text/html,text/html,en,"""Home | The Montgomery News No challenge befor..."
2,20190930201728,themontynews.org,https://www.themontynews.org/single-post/2018/...,text/html,text/html,en,Driver Fired after Kindergartners’ Bus Goes Mi...
3,20190930201743,themontynews.org,https://www.themontynews.org/fullscreen-page,text/html,text/html,en,Fullscreen Page | themontynews No challenge be...
4,20190930201748,themontynews.org,https://www.themontynews.org/home,text/html,text/html,en,"""Home | The Montgomery News No challenge befor..."
5,20190930214858,themontynews.org,https://www.themontynews.org/blog,text/html,text/html,en,"""Blog | The Montgomery News No challenge befor..."
6,20190930225158,themontynews.org,https://www.themontynews.org/advertise,text/html,text/html,en,Advertise | United States | The Montgomery New...
7,20191001000640,themontynews.org,https://www.themontynews.org/calendar,text/html,text/html,en,Calendar | themontynews No challenge before us...
8,20191001005252,themontynews.org,https://www.themontynews.org/obituaries,text/html,text/html,en,Obituaries | themontynews No challenge before ...
9,20191001011404,themontynews.org,https://www.themontynews.org/single-post/2019/...,text/html,text/html,en,Skillman Resident Airlifted to Hospital Follow...


In [9]:
news_df.tail(10)

,crawl_date,domain,url,mime_type_web_server,mime_type_tika,language,content
1103063,20200714020304,communitynews.org,https://communitynews.org/lawrence-gazette/,text/html,text/html,en,Lawrence Gazette - Community News Towns Borden...
1103064,20200714020359,communitynews.org,https://communitynews.org/lawrence-gazette/,text/html,text/html,en,Lawrence Gazette - Community News Towns Borden...
1103065,20200813200315,communitynews.org,https://communitynews.org/lawrence-gazette/,text/html,text/html,en,"""Lawrence Gazette - Community News Towns Borde..."
1103066,20200813200415,communitynews.org,https://communitynews.org/lawrence-gazette/,text/html,text/html,en,"""Lawrence Gazette - Community News Towns Borde..."
1103067,20200912162907,communitynews.org,https://communitynews.org/lawrence-gazette/,text/html,text/html,en,Lawrence Gazette - Community News Towns Borden...
1103068,20200915080728,communitynews.org,https://communitynews.org/lawrence-gazette/,text/html,text/html,en,Lawrence Gazette - Community News Towns Borden...
1103069,20200916164155,communitynews.org,https://communitynews.org/lawrence-gazette/embed/,text/html,text/html,en,Lawrence Gazette - Community News Towns Borden...
1103070,20201013180733,communitynews.org,https://communitynews.org/lawrence-gazette/,text/html,text/html,en,"""Lawrence Gazette - Community News Towns Borde..."
1103071,20201113020609,communitynews.org,https://communitynews.org/lawrence-gazette/,text/html,text/html,en,"""Lawrence Gazette | Community News Towns Borde..."
1103072,20201214102707,communitynews.org,https://communitynews.org/lawrence-gazette/,text/html,text/html,en,Lawrence Gazette - Community News Towns Borden...


In [10]:
print(news_df.shape)
unique_domain = news_df['domain'].unique()
print("Unique domains", len(unique_domain))
unique_url = news_df['url'].unique()
print("Unique urls", len(unique_url))
unique_content = news_df['content'].unique()
print("Unique content", len(unique_content))


(1103073, 7)
Unique domains 230
Unique urls 433155
Unique content 514214


In [12]:
loc_df = pd.read_csv('/Users/gayathri/Documents/nj-local-news-analysis/outlet_location.csv')
loc_df.head()


,"Source of information [ENTER MULTIPLE CODES IF APPLICABLE] 1=Editor&Publisher, 2=Cision, 3=LION, 4=INN, 5=CCM, 6=BIA/Kelsey, 10=Manual search, 11=Lib of Congress (DON'T USE), 12=OnlineNewspaperDirectory, 13=MichelesList, 14=Mondotimes, 15=ABYZ, 16=NAM, 17=NatlNewspaperAssoc, 18=Other, 19 = NJPA",Name of outlet or organization,"Coverage area (9=not applicable, e.g. 5 or more counties",Munis covered (clean for Zev) 9=coverage area not clear or too diffuse,"Municipality ""character"" code; 6 = even mix of characteristics; 9 = n/a (from Nj State Police 2016 Uniform Crime Report: https://www.njsp.org/ucr/2016/index.shtml, section 7)",Outlet/org URL (homepage) (9=no website),"FREQUENCY 1=daily, 2=weekly, 3=biweekly, 4=monthly, 5=bimonthly, 6=regularly (but not daily), 7=irregularly, 8 annually","Original medium 1=Newspaper 2=Television 3=Radio 4=Online, 5=Magazine",Outlet/org owner (9=Don't know or N/A),"Outlet's language of publication 1=English, 2=Spanish, 3=Arabic, 4=Hindi or other language of India, 5=Mandarin or Cantonese, 6=Other, 7=Multilingual, 8=Tagalog or other Filipino language 9=Don't know/NA, 10=Korean, 11=Polish, 12=Ukranian","Does this outlet serve an ethnic community? 1=yes, 2=no","Is this outlet public media? (1=yes, 2=no)","(for Philly and NYC outlets) Does this outlet serve NJ? (1=yes, 2=no, 9=N/A)",Notes,Other potential variables: staff size,other variable: is the outlet hyperlocal? Y/N
0,"15, 2",201,"Bergen County/ Urban (46), Suburban (20), Rura...",Bergen County,Urban,http://www.201magazine.com/#201magazine,8,5,Gannett,1,2,2.0,9,NaN,NaN,NaN
1,"15, 2, 1, 5","(Rutgers-Newark) Observer, The",Newark,Newark,Urban,https://theobservernewark.com/news,2,1,Rutgers University-Newark,1,2,2.0,9,NaN,NaN,NaN
2,"18, 2",07030 Hoboken,Hoboken,Hoboken,Urban,https://hudsonreporter.com/category/issues/070...,5,5,"Packet Media, LLC, Newspaper Media Group",1,2,2.0,9,NaN,NaN,NaN
3,"5, 15, 12, 1, 16, 2",24 horas,Newark,Newark,Urban,https://24horasnewspaper.com/,1,1,Independently owned,6 (Portuguese),1,2.0,9,NaN,NaN,NaN
4,9,AC Primetime,Atlantic City,Atlantic City,Urban,https://acprimetime.com/atlantic-city-primetim...,7,4,Downbeach Digital News Network,1,2,2.0,9,cannot find location or contact info for owner...,NaN,NaN


In [13]:
loc_df.tail(10)


,"Source of information [ENTER MULTIPLE CODES IF APPLICABLE] 1=Editor&Publisher, 2=Cision, 3=LION, 4=INN, 5=CCM, 6=BIA/Kelsey, 10=Manual search, 11=Lib of Congress (DON'T USE), 12=OnlineNewspaperDirectory, 13=MichelesList, 14=Mondotimes, 15=ABYZ, 16=NAM, 17=NatlNewspaperAssoc, 18=Other, 19 = NJPA",Name of outlet or organization,"Coverage area (9=not applicable, e.g. 5 or more counties",Munis covered (clean for Zev) 9=coverage area not clear or too diffuse,"Municipality ""character"" code; 6 = even mix of characteristics; 9 = n/a (from Nj State Police 2016 Uniform Crime Report: https://www.njsp.org/ucr/2016/index.shtml, section 7)",Outlet/org URL (homepage) (9=no website),"FREQUENCY 1=daily, 2=weekly, 3=biweekly, 4=monthly, 5=bimonthly, 6=regularly (but not daily), 7=irregularly, 8 annually","Original medium 1=Newspaper 2=Television 3=Radio 4=Online, 5=Magazine",Outlet/org owner (9=Don't know or N/A),"Outlet's language of publication 1=English, 2=Spanish, 3=Arabic, 4=Hindi or other language of India, 5=Mandarin or Cantonese, 6=Other, 7=Multilingual, 8=Tagalog or other Filipino language 9=Don't know/NA, 10=Korean, 11=Polish, 12=Ukranian","Does this outlet serve an ethnic community? 1=yes, 2=no","Is this outlet public media? (1=yes, 2=no)","(for Philly and NYC outlets) Does this outlet serve NJ? (1=yes, 2=no, 9=N/A)",Notes,Other potential variables: staff size,other variable: is the outlet hyperlocal? Y/N
769,"2, 6",WWOR-TV,9,9,NaN,http://www.my9nj.com,1,2,"Fox Television Stations, LLC",1,2,2.0,9,"licensed in Patterson NJ, serves broader NY ma...",NaN,NaN
770,"2, 16",WWRL-AM,9,9,9,http://radiozindagi.com/newyork/,1,3,Independently owned (Nimisha and Jeetendra Shu...,4,1,2.0,1,technically based in NYC; according to the eth...,NaN,NaN
771,"5, 16, 2",WWRU-AM,9,9,9,https://www.am1660.com/,1,3,"Multicultural Radio Broadcasting Licensee, LLC",10,1,2.0,1,NaN,NaN,NaN
772,"2, 16",WWRV-AM,9,9,9,http://www.radiovision.net/,1,3,Radio Vision Cristiana Management,2,1,2.0,9,NaN,NaN,NaN
773,"15, 6, 5, 16, 2",WWSI-TV,9,9,9,https://www.telemundo62.com/,1,2,Comcast/NBC,2,1,2.0,1,NaN,NaN,NaN
774,"5, 2, 16",WWTR-AM,9,9,9,https://www.ebcmusic.com/,1,3,"EBC Music, Inc.",4,1,2.0,9,NaN,NaN,NaN
775,"5, 2, 6, 16",WXTV-TV,9,9,9,9,1,2,Univision Communications,2,1,2.0,9,NY metro area,NaN,NaN
776,"18, 2",Wyckoff-Franklin Lakes Daily Voice,"Franklin Lakes (Suburban), Oakland (Suburban),...","Franklin Lakes, Oakland, Wyckoff",Suburban,https://dailyvoice.com/new-jersey/wyckoff,1,4,Cantata Media (Jerry DeMarco),1,2,2.0,1,NaN,NaN,NaN
777,"16, 2","Yeshiva World, The",9,9,9,https://www.theyeshivaworld.com/,1,4,Independently owned (Judah (Yehudah) Eckstein),1,1,2.0,9,NaN,NaN,NaN
778,16,Zaman Amerika,9,9,9,http://zamanamerika.com/,1,4,Independently owned (Zaman Ltd.),6 (Turkish),1,2.0,9,NaN,NaN,NaN


In [14]:

unique_outlet = loc_df['Outlet/org URL (homepage) (9=no website)'].unique()
print(len(unique_outlet))

622


In [27]:
outlet_loc = loc_df[['Outlet/org URL (homepage) (9=no website)', 'Munis covered (clean for Zev) 9=coverage area not clear or too diffuse']]
print(outlet_loc.shape)


# Create a dictionary of outlet and location
outlet_loc_dict = {}

for i in range(len(outlet_loc)):
    outlet = outlet_loc.iloc[i, 0]
    loc = outlet_loc.iloc[i, 1]
    loc = loc.split(',')
    loc = [x.strip().lower() for x in loc]
    outlet_loc_dict[outlet] = loc

print(outlet_loc_dict)


(779, 2)
{'http://www.201magazine.com/#201magazine': ['bergen county'], 'https://theobservernewark.com/news': ['newark'], 'https://hudsonreporter.com/category/issues/07030-hoboken-issue/': ['hoboken'], 'https://24horasnewspaper.com/': ['newark'], 'https://acprimetime.com/atlantic-city-primetime-news/': ['atlantic city'], 'http://advertisernewsnorth.com': ['vernon township', 'sussex', 'wantage township'], 'http://advertisernewssouth.com': ['hamburg', 'franklin borough', 'lafayette township', 'ogdensburg'], 'http://africansuntimes.com/': ['9'], 'http://www.afrophilly.com/': ['cherry hill', 'camden', 'hillside'], 'http://www.ahoranews.net': ['9'], 'http://aldianews.com': ['9'], 'http://allaroundpennsauken.com': ['pennsauken'], 'http://amandlanews.com': ['9'], 'http://www.amboyguardian.com/': ['perth amboy', 'south amboy'], 'https://www.americaoggi.us/': ['9'], 'https://americanonewspaper.com/': ['9'], 'https://nepszava.us': ['9'], 'http://www.anointedonline.net/': ['camden'], '9': ['9'], 

In [28]:
# Getting the coverage area / outlet of the news source
# when domain is 10000 there is no valid key found
# unique_outlets - 622, domains - 230
news_source = news_df['domain'][1103072]
print(news_source)
for key in outlet_loc_dict:
    i=0
    if re.search(news_source, key):
        value = outlet_loc_dict[key]
        i = i+1
        break
if i==0:
    print("Not found")
else:
    print(key,value)
    print(i)



communitynews.org
https://communitynews.org/bordentown-current/ ['bordentown']
1


In [29]:
def get_geolocation(loc_name):
    '''
    This function takes in a location name and returns the latitude and longitude of the location
    '''
    location = geolocator.geocode(loc_name, components={ "administrative_area": "NJ", "country": "US"})
    return (location.latitude, location.longitude) if location else None

In [30]:
# total number of unique coverage_areas and the number of existing coverage areas
outlet_lat_long_mapping = {}
unique_coverage_areas = set()
i=0
for outlet, coverage_areas in outlet_loc_dict.items():
    if outlet !='9' and coverage_areas[0] != '9':
        i = i + 1
        
        unique_coverage_areas.add(coverage_areas[0])

print(i)        
print(len(unique_coverage_areas))

    

490
262


In [31]:
# For storing the outlet and its location (latitude and longitude)
outlet_lat_long_mapping = {}

for outlet, coverage_areas in outlet_loc_dict.items():
    if outlet !='9' and coverage_areas[0] != '9':
        lat_long = get_geolocation(coverage_areas[0])    #getting the lat long for the first coverage_area for each outlet
        if lat_long:
            outlet_lat_long_mapping[outlet] = lat_long

print(outlet_lat_long_mapping)    

{'http://www.201magazine.com/#201magazine': (40.9262762, -74.07701), 'https://theobservernewark.com/news': (40.735657, -74.1723667), 'https://hudsonreporter.com/category/issues/07030-hoboken-issue/': (40.7439905, -74.0323626), 'https://24horasnewspaper.com/': (40.735657, -74.1723667), 'https://acprimetime.com/atlantic-city-primetime-news/': (39.3642834, -74.4229266), 'http://advertisernewsnorth.com': (41.1985786, -74.4828866), 'http://advertisernewssouth.com': (41.1534294, -74.5762713), 'http://www.afrophilly.com/': (39.926813, -75.0246312), 'http://allaroundpennsauken.com': (39.9562052, -75.058009), 'http://www.amboyguardian.com/': (40.5067723, -74.2654234), 'http://www.anointedonline.net/': (39.9259463, -75.1196199), 'http://arabvoice.com/': (40.9167654, -74.17181099999999), 'http://www.stocktonargo.com': (39.4928235, -74.5596879), 'http://www.app.com/': (40.3470543, -74.0643065), 'http://asburyparksun.com/': (40.2203907, -74.0120817), 'http://www.ahherald.com/': (40.2589455, -74.123

In [32]:
# Exporting outlet and its location (latitude and longitude) to a csv file

output_file = 'outlet_lat_long_mapping.csv'

with open(output_file, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(['Outlet URL', 'Latitude', 'Longitude'])
    for outlet, lat_long in outlet_lat_long_mapping.items():
        csv_writer.writerow([outlet, lat_long[0], lat_long[1]])

print(f"Output written to {output_file}")


Output written to outlet_lat_long_mapping.csv


In [19]:
#Method to remove the common header from the given content
def modify_content(sample_content):
    modified_content = sample_content.replace('The Montgomery News The Voice of Your Community — Serving Belle Mead, Blawenburg, Griggstown, Harlingen, Montgomery, Skillman, and Rocky Hill, NJ', '') 
    return modified_content

In [33]:
# Test code to extract GPEs from a news article

newsContent = modify_content(news_df['content'][1000])
print(newsContent)
print(news_df['url'][1000])

doc_1 = nlp(newsContent)
#print(doc_1.ents)

for ent in doc_1.ents:
    if ent.label_ == 'GPE':
        print(ent.text, "|", ent.label_, "|", spacy.explain(ent.label_))

Single Post | themontynews No challenge before us is more important and more potentially life-giving than that we come to see and know our fellow citizens, our neighbors who’ve become strangers. — Krista Tippett  Home News Stories Advertise Calendar Obituaries More... I'm busy working on my blog posts. Watch this space! Please reload Recent Posts Montgomery Planning Board Approves Chase Bank Bran... October 8, 2019 Rocky Hill’s Pacific Southern Model Railroad Show... October 7, 2019 Rocky Hill Day —October 5 October 5, 2019 Please reload Archive October 2019 September 2019 August 2019 July 2019 June 2019 May 2019 April 2019 March 2019 February 2019 January 2019 December 2018 November 2018 October 2018 September 2018 August 2018 July 2018 June 2018 May 2018 April 2018 March 2018 February 2018 January 2018 Please reload Tags Alfred Hadinger Alice Chen Augusta National Barbara Preston Belle Mead Belle Mead NJ Brendan Patrick Welch
https://www.themontynews.org/single-post/2019/10/08/00f0d6

In [22]:
#Counting the occurence of each GPE in the Content retrieved
from collections import Counter

paragraph = """Yesterday, I decided to take a tour around New Jersey and explore its wonderful cities. The first city on my list was Newark. I had always heard great things about Newark, so I was excited to see it for myself. The vibe was welcoming and I decided to stay overnight before heading off to Trenton. Trenton had a historical allure that captivated me. I couldn't resist revisiting Trenton the next day to explore more. Then I decided to travel down the coast to Atlantic City. The bustling life of Atlantic City was a stark contrast to the tranquility of Trenton.

As I left Atlantic City, I started driving towards Paterson. I was greeted with the cultural diversity of Paterson, which was an experience in itself. My next stop was Elizabeth. Elizabeth was a beautiful city with the best parks I've ever visited. From Elizabeth, I went to visit Princeton. The academic vibe in Princeton was very refreshing. I was so taken by the charm of Princeton that I decided to spend the entire next day there as well.

I then headed up north to visit Clifton. I enjoyed the cityscape views from the hills of Clifton so much that I went there again the following day. After Clifton, I visited Camden and was amazed by the transformation of the city. The next day, I went back to Camden to attend a local music festival. Finally, I decided to end my journey in Hoboken. The view of New York City from Hoboken was breathtaking. I spent another day in Hoboken just to enjoy this view a little more."""



doc_2 = nlp(paragraph)

gpe_ents = [ent.text for ent in doc_2.ents if ent.label_ == 'GPE']
print(gpe_ents)

gpe_counts = Counter(gpe_ents)
for gpe, count in gpe_counts.items():
    print(f"{gpe}: {count}")
# for ent in doc_2.ents:
#     if ent.label_ == 'GPE':
#         print(ent.text, "|", ent.label_, "|", spacy.explain(ent.label_))

['New Jersey', 'Newark', 'Newark', 'Trenton', 'Trenton', 'Atlantic City', 'Atlantic City', 'Trenton', 'Atlantic City', 'Princeton', 'Princeton', 'Camden', 'Camden', 'Hoboken', 'New York City', 'Hoboken', 'Hoboken']
New Jersey: 1
Newark: 2
Trenton: 3
Atlantic City: 3
Princeton: 2
Camden: 2
Hoboken: 3
New York City: 1


In [23]:
#Retriving the indexes of all the columns where the url matches themontynews.
index = news_df[news_df['url'] == "https://www.themontynews.org/single-post/2019/06/11/Sam-P-Panicaro-98"].index
print(index)
print(news_df.shape)

Int64Index([693, 1652, 2534, 3472, 4408, 5197, 6225, 8098, 8717], dtype='int64')
(1103073, 7)


In [34]:
outlet_lat_long = None
news_articles_info = {}
news_url = news_df['url'][2345]
news_source = news_df['domain'][2345]
news_source_pattern = re.compile(re.escape(news_source), re.IGNORECASE)
# print("news source pattern")
print(news_source)
for key, lat_long in outlet_lat_long_mapping.items():
    if news_source_pattern.search(key):
        print("key",key)
        outlet_lat_long = lat_long
        print("outlet_lat_long",outlet_lat_long)
        break

if outlet_lat_long is None:
    print("Outlet not found in the mapping.")
else:
    test_content = modify_content(news_df['content'][2345])
    doc = nlp(test_content)

    gpe_locations = {}

    for ent in doc.ents:
        if ent.label_ == 'GPE':
            gpe_text = ent.text
            print("Locations found in content",gpe_text)

            # Skip if GPE is a country
            try:
                location = geolocator.geocode(gpe_text)
                if location and location.raw.get('type') == 'country':
                    continue
            except (GeocoderTimedOut, GeocoderServiceError):
                print(f"Geocoder service error occurred while processing: {gpe_text}")
                continue

            if gpe_text not in gpe_locations:
                # Get all locations for the GPE within the same region as the outlet
                print("new_gpe_text",gpe_text)
                gpe_locations_within_region = geolocator.geocode(gpe_text, components={"administrative_area": "NJ", "country": "US"}, exactly_one=False)
                print("gpe_locations_within_region",gpe_locations_within_region)
                closest_location = None
                min_distance = float('inf')
                print("UPDATING MIN_DISTANCE")

                # Iterate through all found locations for each GPE and find the closest one to the outlet
                if gpe_locations_within_region is not None:
                    print("iterating",gpe_locations_within_region )
                    for loc in gpe_locations_within_region:
                        print("loc",loc)
                        loc_lat_long = (loc.latitude, loc.longitude)
                        print(loc_lat_long, outlet_lat_long)
                        distance = great_circle(outlet_lat_long, loc_lat_long).miles
                        print("DISTANCE IS",distance)

                        if distance < min_distance:
                            print("UPDATE CLOSEST DISTANCE.....")
                            min_distance = distance
                            closest_location = loc_lat_long
                            print("closest_location",closest_location)

                    # Associate the GPE with its closest location to the outlet
                    # gpe_locations[gpe_text] = {"outlet": news_source, "lat_long": closest_location}
                    gpe_locations[gpe_text] = closest_location
                    print("gpe_locations[gpe_text]",gpe_locations[gpe_text])
   
    news_articles_info[news_url] = {
        "outlet": news_source,
        "outlet_lat_long": outlet_lat_long,
        "GPEs": gpe_locations
    }            

    print("news_articles_info")
    print(news_articles_info)
                            


themontynews.org
key https://www.themontynews.org/
outlet_lat_long (40.4675835, -74.6628114)
news_articles_info
{'https://www.themontynews.org/single-post/2019/09/17/00f0d6_dfb5c04b44f4bdf19db0520fa84d6fc6_1266.json': {'outlet': 'themontynews.org', 'outlet_lat_long': (40.4675835, -74.6628114), 'GPEs': {}}}


In [35]:
outlet_lat_long = None
news_articles_info = {}
news_url = news_df['url'][212]
news_source = news_df['domain'][212]
news_source_pattern = re.compile(re.escape(news_source), re.IGNORECASE)
print("news_source",news_source)
for key, lat_long in outlet_lat_long_mapping.items():
    if news_source_pattern.search(key):
        print("key",key)
        outlet_lat_long = lat_long
        print("lat_long",outlet_lat_long)
        break

if outlet_lat_long is None:
    print("Outlet not found in the mapping.")
else:
    test_content = modify_content(news_df['content'][212])
    doc = nlp(test_content)

    gpe_locations = {}

    for ent in doc.ents:
        if ent.label_ == 'GPE':
            gpe_text = ent.text
            print("Locations found in content")
            print(gpe_text)

            # Skip if GPE is a country
            try:
                location = geolocator.geocode(gpe_text)
                if location and location.raw.get('type') == 'country':
                    continue
            except (GeocoderTimedOut, GeocoderServiceError):
                print(f"Geocoder service error occurred while processing: {gpe_text}")
                continue

            if gpe_text not in gpe_locations:
                # Get all locations for the GPE within the same region as the outlet
                print("gpe_text")
                print(gpe_text)
                gpe_locations_within_region = geolocator.geocode(gpe_text, components={"administrative_area": "NJ", "country": "US"}, exactly_one=False)
                closest_location = None
                min_distance = float('inf')
                print("UPDATING MIN_DISTANCE")
                
                # Iterate through all found locations for each GPE and find the closest one to the outlet
                if gpe_locations_within_region is not None:
                    print(gpe_locations_within_region)
                    for loc in gpe_locations_within_region:
                        print("loc")
                        print(loc)
                        loc_lat_long = (loc.latitude, loc.longitude)
                        print(loc_lat_long, outlet_lat_long)
                        distance = great_circle(outlet_lat_long, loc_lat_long).miles

                        if distance < min_distance:
                            min_distance = distance
                            closest_location = loc_lat_long
                            print("closest_location")
                            print(closest_location)

                    # Associate the GPE with its closest location to the outlet
                    # gpe_locations[gpe_text] = {"outlet": news_source, "lat_long": closest_location}
                    gpe_locations[gpe_text] = closest_location  
                    print("Adding the value to gpe_locations.......")
   
    news_articles_info[news_url] = {
        "outlet": news_source,
        "outlet_lat_long": outlet_lat_long,
        "GPEs": gpe_locations
    }            

    print("news_articles_info")
    print(news_articles_info)




news_source themontynews.org
key https://www.themontynews.org/
lat_long (40.4675835, -74.6628114)
news_articles_info
{'https://www.themontynews.org/single-post/2019/09/11/00f0d6_83da7b2652d601211b4640a0426c4bec_1258.json': {'outlet': 'themontynews.org', 'outlet_lat_long': (40.4675835, -74.6628114), 'GPEs': {}}}


In [36]:
# Reading all articles for monty news:
loc_store = {}
loc_store_file = 'loc_store.csv'

country_store = {}
country_store_file = 'country_store.csv'

try:
    loc_store_df = pd.read_csv(loc_store_file)
    loc_store = loc_store_df.set_index("gpe").T.to_dict('list')

    country_store_df = pd.read_csv(country_store_file)
    country_store = {country: None for country in country_store_df['country']}

except FileNotFoundError:
    loc_store = {}
    country_store = {}

# List to store the final data
news_articles_info = []


# Filter the DataFrame to include only the rows where the domain is 'themontynews.org'
filtered_df = news_df[news_df['domain'] == 'themontynews.org'].sample(1000)

for _, row in filtered_df.iterrows():
    news_url = row['url']
    news_source = row['domain']
    news_source_pattern = re.compile(re.escape(news_source), re.IGNORECASE)

    outlet_lat_long = None
    for key, lat_long in outlet_lat_long_mapping.items():
        if news_source_pattern.search(key):
            outlet_lat_long = lat_long
            break

    if outlet_lat_long is None:
        print("Outlet not found in the mapping.")
        continue

    test_content = modify_content(row['content'])
    doc = nlp(test_content)

    gpe_locations = {}
    gpe_occurrences = {} # Dictionary to store GPE occurrences


    for ent in doc.ents:
        if ent.label_ == 'GPE':
            gpe_text = ent.text

            gpe_occurrences[gpe_text] = gpe_occurrences.get(gpe_text, 0) + 1    
           
            # try:
            #     location = geolocator.geocode(gpe_text)
            #     if location and location.raw.get('type') == 'country':
            #         country_store[gpe_text] = None
            #         continue
            # except (GeocoderTimedOut, GeocoderServiceError):
            #     print(f"Geocoder service error occurred while processing: {gpe_text}")
            #     continue
            
            if gpe_text not in gpe_locations:

                if gpe_text in loc_store:   # If the GPE exists in loc_store, use it
                    closest_location = loc_store[gpe_text]
                    gpe_locations[gpe_text] = closest_location
                    print("Saved geolocator call")
                    continue


                # Skip if GPE is a country
                if gpe_text in country_store:
                    print(f"Found country in dict: {gpe_text}")
                    continue
                else:
                    try:
                        location = geolocator.geocode(gpe_text)
                        print("geolocator called for country check", location)
                        if location and 'country' in location.raw['address_components'][0]['types']:
                            print(f"Found country: {gpe_text}")
                            country_store[gpe_text] = None
                            continue
                    except (GeocoderTimedOut, GeocoderServiceError):
                        print(f"Geocoder service error occurred while processing: {gpe_text}")
                        continue
                
                # Get all locations for the GPE within the same region as the outlet
                gpe_locations_within_region = geolocator.geocode(gpe_text, components={"administrative_area": "NJ", "country": "US"}, exactly_one=False)
                print("geolocator called")
                print('gpe_locations_within_region', gpe_locations_within_region)
                closest_location = None
                min_distance = float('inf')

                # Iterate through all found locations for each GPE and find the closest one to the outlet
                if gpe_locations_within_region is not None:
                    for loc in gpe_locations_within_region:
                        print('gpe_text, loc:', gpe_text, loc)
                        
                        loc_lat_long = (loc.latitude, loc.longitude)
                        loc_store[gpe_text] = loc_lat_long
                        distance = great_circle(outlet_lat_long, loc_lat_long).miles

                        if distance < min_distance:
                            min_distance = distance
                            closest_location = loc_lat_long

                    # Associate the GPE with its closest location to the outlet
                    gpe_locations[gpe_text] = closest_location

    # Store the data for each GPE
    for gpe, gpe_lat_long in gpe_locations.items():
        news_articles_info.append({
            "news_url": news_url,
            "outlet": news_source,
            "outlet_latitude": outlet_lat_long[0],
            "outlet_longitude": outlet_lat_long[1],
            "gpe": gpe,
            "gpe_latitude": gpe_lat_long[0] if gpe_lat_long else None,
            "gpe_longitude": gpe_lat_long[1] if gpe_lat_long else None,
            "gpe_occurrences": gpe_occurrences[gpe]
        })

print(news_articles_info)
print('loc_store', loc_store)
print('country_store', country_store)
# Convert the list of dictionaries to a DataFrame
news_articles_df = pd.DataFrame(news_articles_info)

# Export to CSV
news_articles_df.to_csv("news_articles_info_v4.csv", index=False)

# Convert the updated loc_store to a DataFrame and save it to a CSV
loc_store_df = pd.DataFrame.from_dict(loc_store, orient='index', columns=['latitude', 'longitude'])
loc_store_df.reset_index(level=0, inplace=True)
loc_store_df.columns = ['gpe', 'latitude', 'longitude']
loc_store_df.to_csv(loc_store_file, index=False)


country_store_df = pd.DataFrame(list(country_store.keys()), columns=['country'])
country_store_df.to_csv(country_store_file, index=False)

Saved geolocator call
Saved geolocator call
Saved geolocator call
Saved geolocator call
Saved geolocator call
geolocator called for country check Utica, NY, USA
geolocator called
gpe_locations_within_region None
geolocator called for country check None
geolocator called
gpe_locations_within_region None
Saved geolocator call
Saved geolocator call
Saved geolocator call
Saved geolocator call
Saved geolocator call
Saved geolocator call
Saved geolocator call
Saved geolocator call
Saved geolocator call
Saved geolocator call
Saved geolocator call
Saved geolocator call
Saved geolocator call
Saved geolocator call
Saved geolocator call
Saved geolocator call
geolocator called for country check New York, NY, USA
geolocator called
gpe_locations_within_region None
geolocator called for country check 3959 W 165th St, New York, NY 10032, USA
geolocator called
gpe_locations_within_region None
Saved geolocator call
Saved geolocator call
Saved geolocator call
Saved geolocator call
Saved geolocator call
S

In [37]:
try:
    location = geolocator.geocode("Madrid, Spain")
    print("geolocator called for country check", location.raw)
    print('country' in location.raw['address_components'][0]['types'] )
    if location and 'country' in location.raw['address_components'][0]['types'] :
        print(f"Found country: {location}")
        # country_store[gpe_text] = None
except (GeocoderTimedOut, GeocoderServiceError):
    print(f"Geocoder service error occurred while processing: {gpe_text}")

geolocator called for country check {'address_components': [{'long_name': 'Madrid', 'short_name': 'Madrid', 'types': ['locality', 'political']}, {'long_name': 'Madrid', 'short_name': 'M', 'types': ['administrative_area_level_2', 'political']}, {'long_name': 'Community of Madrid', 'short_name': 'MD', 'types': ['administrative_area_level_1', 'political']}, {'long_name': 'Spain', 'short_name': 'ES', 'types': ['country', 'political']}], 'formatted_address': 'Madrid, Spain', 'geometry': {'bounds': {'northeast': {'lat': 40.5638447, 'lng': -3.5249115}, 'southwest': {'lat': 40.3120639, 'lng': -3.8341618}}, 'location': {'lat': 40.4167754, 'lng': -3.7037902}, 'location_type': 'APPROXIMATE', 'viewport': {'northeast': {'lat': 40.5638447, 'lng': -3.5249115}, 'southwest': {'lat': 40.3120639, 'lng': -3.8341618}}}, 'place_id': 'ChIJgTwKgJcpQg0RaSKMYcHeNsQ', 'types': ['locality', 'political']}
False


In [55]:
# DO NOT RUN THIS CODE. Not to be used

# This code identies the common GPEs for each news articles and stores there address details 

import re
from collections import Counter

# Dictionary to store the final data
news_articles_info = {}

# Dictionary to count all GPEs for each outlet
outlet_gpe_counts = Counter()

# Filter the DataFrame to include only the rows where the domain is 'themontynews.org' and randomly select 10 rows
filtered_df = news_df[news_df['domain'] == 'themontynews.org'].sample(100)

for _, row in filtered_df.iterrows():
    news_url = row['url']
    news_source = row['domain']
    news_source_pattern = re.compile(re.escape(news_source), re.IGNORECASE)

    outlet_lat_long = None
    for key, lat_long in outlet_lat_long_mapping.items():
        if news_source_pattern.search(key):
            outlet_lat_long = lat_long
            break

    if outlet_lat_long is None:
        print("Outlet not found in the mapping.")
        continue  

    test_content = row['content']
    doc = nlp(test_content)

    gpe_locations = {}
    gpe_counter = Counter()

    for ent in doc.ents:
        if ent.label_ == 'GPE':
            gpe_text = ent.text

            if gpe_text not in gpe_locations:
                # Get all locations for the GPE within the same region as the outlet
                gpe_locations_within_region = geolocator.geocode(gpe_text, components={"administrative_area": "NJ", "country": "US"}, exactly_one=False)

                closest_location = None
                min_distance = float('inf')

                # Iterate through all found locations for each GPE and find the closest one to the outlet
                if gpe_locations_within_region:
                    for loc in gpe_locations_within_region:
                        loc_lat_long = (loc.latitude, loc.longitude)
                        distance = great_circle(outlet_lat_long, loc_lat_long).miles

                        if distance < min_distance:
                            min_distance = distance
                            closest_location = loc_lat_long

                    # Associate the GPE with its closest location to the outlet
                    gpe_locations[gpe_text] = closest_location

            # Update the GPE counter
            gpe_counter[gpe_text] += 1

    # Update the outlet's count for all GPEs
    for gpe, count in gpe_counter.items():
        outlet_gpe_counts[news_source, gpe] += count

    news_articles_info[news_url] = {
        "outlet": news_source,
        "outlet_lat_long": outlet_lat_long,
        "GPEs": gpe_locations
    }

# Print the outlet and GPEs with their counts
# for (outlet, gpe), count in outlet_gpe_counts.items():
#     print(f"The outlet '{outlet}' mentioned the GPE '{gpe}' {count} times.")

print(news_articles_info)


{'https://www.themontynews.org/judith': {'outlet': 'themontynews.org', 'outlet_lat_long': (40.4675835, -74.6628114), 'GPEs': {'Blawenburg': (40.4076607, -74.7022475), 'Griggstown': (40.4378831, -74.6132128), 'Montgomery': (40.4329141, -74.66346399999999), 'Hammond': (39.6365056, -74.8023853), 'LA': (40.0583238, -74.4056612), 'Montgomery Township': (40.4329141, -74.66346399999999), 'Belle Mead': (40.4675835, -74.6628114)}}, 'https://www.themontynews.org/single-post/2019/12/27/00f0d6_ff4e57c8ff219924c175d017b710d932_1421.json': {'outlet': 'themontynews.org', 'outlet_lat_long': (40.4675835, -74.6628114), 'GPEs': {'Blawenburg': (40.4076607, -74.7022475), 'Griggstown': (40.4378831, -74.6132128), 'Montgomery': (40.4329141, -74.66346399999999)}}, 'https://www.themontynews.org/single-post/2019/09/12/Rocky-Hill-Clam-Smugglers-Return': {'outlet': 'themontynews.org', 'outlet_lat_long': (40.4675835, -74.6628114), 'GPEs': {'Blawenburg': (40.4076607, -74.7022475), 'Griggstown': (40.4378831, -74.6132

In [56]:
# DO NOT RUN THIS CODE. Not to be used
import re
import pandas as pd
from collections import Counter

# Dictionary to store the final data
news_articles_info = []

# Dictionary to count all GPEs for each outlet
outlet_gpe_counts = Counter()

# Filter the DataFrame to include only the rows where the domain is 'themontynews.org' and randomly select 10 rows
filtered_df = news_df[news_df['domain'] == 'themontynews.org'].sample(10)

for _, row in filtered_df.iterrows():
    gpe_counter = Counter()
    news_url = row['url']
    news_source = row['domain']
    news_source_pattern = re.compile(re.escape(news_source), re.IGNORECASE)

    outlet_lat_long = None
    for key, lat_long in outlet_lat_long_mapping.items():
        if news_source_pattern.search(key):
            outlet_lat_long = lat_long
            break

    if outlet_lat_long is None:
        print("Outlet not found in the mapping.")
        continue  

    test_content = row['content']
    doc = nlp(test_content)

    gpe_locations = {}
    

    for ent in doc.ents:
        if ent.label_ == 'GPE':
            gpe_text = ent.text

            # Skip if GPE is a country
            try:
                location = geolocator.geocode(gpe_text)
                if location and location.raw.get('type') == 'country':
                    continue
            except (GeocoderTimedOut, GeocoderServiceError):
                print(f"Geocoder service error occurred while processing: {gpe_text}")
                continue

            if gpe_text not in gpe_locations:
                # Get all locations for the GPE within the same region as the outlet
                gpe_locations_within_region = geolocator.geocode(gpe_text, components={"administrative_area": "NJ", "country": "US"}, exactly_one=False)

                closest_location = None
                min_distance = float('inf')

                # Iterate through all found locations for each GPE and find the closest one to the outlet
                if gpe_locations_within_region:
                    for loc in gpe_locations_within_region:
                        loc_lat_long = (loc.latitude, loc.longitude)
                        distance = great_circle(outlet_lat_long, loc_lat_long).miles

                        if distance < min_distance:
                            min_distance = distance
                            closest_location = loc_lat_long

                    # Associate the GPE with its closest location to the outlet
                    gpe_locations[gpe_text] = closest_location

            # Update the GPE counter
            gpe_counter[gpe_text] += 1

    # Update the outlet's count for all GPEs
    for gpe, count in gpe_counter.items():
        # Skip if GPE is not in gpe_locations
        if gpe not in gpe_locations or gpe not in test_content:
            continue
        outlet_gpe_counts[news_source, gpe] += count

        # Store the data for each GPE
        gpe_lat_long = gpe_locations[gpe]
        news_articles_info.append({
            "news_url": news_url,
            "outlet": news_source,
            "outlet_latitude": outlet_lat_long[0],
            "outlet_longitude": outlet_lat_long[1],
            "gpe": gpe,
            "gpe_latitude": gpe_lat_long[0] if gpe_lat_long else None,
            "gpe_longitude": gpe_lat_long[1] if gpe_lat_long else None,
        })

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(news_articles_info)

# Export the DataFrame to a CSV file
df.to_csv('news_articles_info_1.csv', index=False)

# Print the outlet and GPE with the maximum count
max_outlet_gpe, max_count = outlet_gpe_counts.most_common(1)[0]
print(f"The outlet and GPE with the maximum count of {max_count} is: {max_outlet_gpe}")



The outlet and GPE with the maximum count of 10 is: ('themontynews.org', 'Blawenburg')


In [57]:
def calc_distance(lat1, lng1, lat2, lng2):
    coords_1 = (lat1, lng1)
    coords_2 = (lat2, lng2)
    
    try:
        dist = geodesic(coords_1, coords_2).miles #in miles
    except:
        dist = geopy.distance.great_circle(coords_1, coords_2).miles
        
    return dist
      

In [ ]:
lat1 = 40.494430
lng1 = -74.446350

lat2 = 38.835289
lng2 = -77.308418

print(calc_distance(lat1, lng1, lat2, lng2))



In [ ]:
from geopy.geocoders import GoogleV3

geolocator = GoogleV3(api_key="...")
location = geolocator.geocode("Skillman, NJ")
print(location.address)
print(location.latitude, location.longitude)

818 E Elizabeth St, Fort Collins, CO 80524, USA
40.5745429 -105.0619448


In [ ]:
from geopy.point import Point

# Define the bounding box for the New Jersey region
sw = Point(38.928212, -75.560315)
ne = Point(41.357633, -73.894883)
bounds = (sw, ne)


# Initialize the geolocator with the GoogleV3 geocoder and the bounds parameter


# Search for Berlin within the specified bounds
location = geolocator.geocode("Skillman", components={ "administrative_area": "NJ", "country": "US"})

# Print the address and coordinates of the location
print(location.address)
print((location.latitude, location.longitude))

Skillman, Montgomery, NJ, USA
(40.4201165, -74.7146817)


In [38]:
print(filtered_df.shape)
filtered_df.tail(10)

(1000, 7)


,crawl_date,domain,url,mime_type_web_server,mime_type_tika,language,content
1958,20191012184131,themontynews.org,https://www.themontynews.org/single-post/2019/...,text/html,text/html,en,Single Post | themontynews No challenge before...
7411,20191213104808,themontynews.org,https://www.themontynews.org/single-post/2019/...,text/html,text/html,en,Single Post | themontynews No challenge before...
1632,20191009000400,themontynews.org,https://www.themontynews.org/single-post/2019/...,text/html,text/html,en,"Evelyn Gayle Long Flora, 77 No challenge befor..."
3634,20191020212730,themontynews.org,https://www.themontynews.org/single-post/2019/...,text/html,text/html,en,"Irwin S. Zonis, 88 No challenge before us is m..."
7803,20191213150902,themontynews.org,https://www.themontynews.org/single-post/2019/...,text/html,text/html,en,Single Post | themontynews No challenge before...
8005,20191213165704,themontynews.org,https://www.themontynews.org/single-post/2018/...,text/html,text/html,en,Montgomery Synagogue Invites Local Jews and No...
6896,20191111205435,themontynews.org,https://www.themontynews.org/single-post/2019/...,text/html,text/html,en,Single Post | themontynews No challenge before...
6296,20191102054759,themontynews.org,https://www.themontynews.org/single-post/2019/...,text/html,text/html,en,Montgomery Golfer to Compete in Augusta Nation...
1061,20191008203237,themontynews.org,https://www.themontynews.org/single-post/2019/...,text/html,text/html,en,Single Post | themontynews No challenge before...
3370,20191020205258,themontynews.org,https://www.themontynews.org/single-post/2019/...,text/html,text/html,en,Single Post | themontynews No challenge before...


In [39]:
#test_content = modify_content(news_df['content'][1040])
test_content = news_df['content'][3060]

print(test_content)
doc = nlp(test_content)
ec = set()
i=0
for ent in doc.ents:
    #print(ent,ent.label_)
    if ent.label_ == 'GPE':
        i=i+1
        gpe_text = ent.text
        print("Locations found in content")
        print(gpe_text)
        ec.add(gpe_text)

print(i)

Single Post | themontynews No challenge before us is more important and more potentially life-giving than that we come to see and know our fellow citizens, our neighbors who’ve become strangers. — Krista Tippett The Montgomery News The Voice of Your Community — Serving Belle Mead, Blawenburg, Griggstown, Harlingen, Montgomery, Skillman, and Rocky Hill, NJ Home News Stories Advertise Calendar Obituaries More... I'm busy working on my blog posts. Watch this space! Please reload Recent Posts Groundbreaking for 86-Unit Affordable Apartment Bu... October 19, 2019 Route 518 Rocky Hill Road Work Scheduled Oct 23 —... October 15, 2019 Montgomery Planning Board Approves Chase Bank Bran... October 8, 2019 Please reload Archive October 2019 September 2019 August 2019 July 2019 June 2019 May 2019 April 2019 March 2019 February 2019 January 2019 December 2018 November 2018 October 2018 September 2018 August 2018 July 2018 June 2018 May 2018 April 2018 March 2018 February 2018 January 2018 Please re

In [40]:
nlp = spacy.load('en_core_web_lg')
patterns = [{"label": "GPE", "pattern": "Skillman"}]
#test_content = modify_content(news_df['content'][1040])
test_content = news_df['content'][3060]
print(test_content)
uc = set()
ruler = nlp.add_pipe("entity_ruler","ruleActions", config={"overwrite_ents": True})
ruler.add_patterns(patterns)

print(f'spaCy Pipelines: {nlp.pipe_names}')

doc = nlp(test_content)
j=0
    
for ent in doc.ents:
    #print(ent, ent.label_)
    if ent.label_ == 'GPE':
        j = j+1
        gpe_text = ent.text
        print("Locations found in content:")
        print(gpe_text)
        uc.add(gpe_text)

print(j)

Single Post | themontynews No challenge before us is more important and more potentially life-giving than that we come to see and know our fellow citizens, our neighbors who’ve become strangers. — Krista Tippett The Montgomery News The Voice of Your Community — Serving Belle Mead, Blawenburg, Griggstown, Harlingen, Montgomery, Skillman, and Rocky Hill, NJ Home News Stories Advertise Calendar Obituaries More... I'm busy working on my blog posts. Watch this space! Please reload Recent Posts Groundbreaking for 86-Unit Affordable Apartment Bu... October 19, 2019 Route 518 Rocky Hill Road Work Scheduled Oct 23 —... October 15, 2019 Montgomery Planning Board Approves Chase Bank Bran... October 8, 2019 Please reload Archive October 2019 September 2019 August 2019 July 2019 June 2019 May 2019 April 2019 March 2019 February 2019 January 2019 December 2018 November 2018 October 2018 September 2018 August 2018 July 2018 June 2018 May 2018 April 2018 March 2018 February 2018 January 2018 Please re

In [41]:
print("Before er", ec)
print("After er", uc)

Before er {'Montgomery', 'Blawenburg', 'Griggstown'}
After er {'Somerset County', 'Rocky Hill', 'Somerset', 'NJ', 'Augusta', 'Griggstown', 'Blawenburg', 'Skillman', 'Harlingen', 'Montgomery'}


In [42]:
filtered_df.head(20)

,crawl_date,domain,url,mime_type_web_server,mime_type_tika,language,content
2861,20191020193754,themontynews.org,https://www.themontynews.org/single-post/2019/...,text/html,text/html,en,Single Post | themontynews No challenge before...
5568,20191102015858,themontynews.org,https://www.themontynews.org/single-post/2019/...,text/html,text/html,en,Single Post | themontynews No challenge before...
3775,20191024202014,themontynews.org,https://www.themontynews.org/single-post/2019/...,text/html,text/html,en,Single Post | themontynews No challenge before...
4172,20191024214032,themontynews.org,https://www.themontynews.org/single-post/2019/...,text/html,text/html,en,Single Post | themontynews No challenge before...
1702,20191009002826,themontynews.org,https://www.themontynews.org/single-post/2018/...,text/html,text/html,en,"After 34 years, Beloved Attorney Retires with ..."
1250,20191008215732,themontynews.org,https://www.themontynews.org/single-post/2019/...,text/html,text/html,en,Single Post | themontynews No challenge before...
2678,20191012204521,themontynews.org,https://www.themontynews.org/single-post/2018/...,text/html,text/html,en,"""Montgomery Pathways No challenge before us is..."
2668,20191012204355,themontynews.org,https://www.themontynews.org/single-post/2018/...,text/html,text/html,en,"Dorothy A. Slusarchyk, 79 No challenge before ..."
6589,20191102064928,themontynews.org,https://www.themontynews.org/single-post/2019/...,text/html,text/html,en,Single Post | themontynews No challenge before...
3030,20191020200356,themontynews.org,https://www.themontynews.org/single-post/2019/...,text/html,text/html,en,Single Post | themontynews No challenge before...


In [43]:
filtered_df.tail(20)

,crawl_date,domain,url,mime_type_web_server,mime_type_tika,language,content
6932,20191111210110,themontynews.org,https://www.themontynews.org/single-post/2019/...,text/html,text/html,en,Single Post | themontynews No challenge before...
5368,20191030211723,themontynews.org,https://www.themontynews.org/single-post/2019/...,text/html,text/html,en,Single Post | themontynews No challenge before...
8461,20200113161653,themontynews.org,https://www.themontynews.org/single-post/2019/...,text/html,text/html,en,Single Post | themontynews No challenge before...
1096,20191008204416,themontynews.org,https://www.themontynews.org/single-post/2019/...,text/html,text/html,en,Single Post | themontynews No challenge before...
5878,20191102035207,themontynews.org,https://www.themontynews.org/single-post/2019/...,text/html,text/html,en,Single Post | themontynews No challenge before...
5403,20191030212202,themontynews.org,https://www.themontynews.org/single-post/2019/...,text/html,text/html,en,Single Post | themontynews No challenge before...
6059,20191102044531,themontynews.org,https://www.themontynews.org/single-post/2019/...,text/html,text/html,en,Single Post | themontynews No challenge before...
4995,20191030201732,themontynews.org,https://www.themontynews.org/single-post/2019/...,text/html,text/html,en,Single Post | themontynews No challenge before...
5741,20191102030502,themontynews.org,https://www.themontynews.org/single-post/2019/...,text/html,text/html,en,Single Post | themontynews No challenge before...
2603,20191012203439,themontynews.org,https://www.themontynews.org/single-post/2018/...,text/html,text/html,en,"Jeffrey A. Skaar, 53 No challenge before us is..."


In [44]:
#Samples used to analyse the header or footer
content_1 = news_df['content'][6135]
content_1_url = news_df['url'][6135]
content_2 = news_df['content'][700]
content_3 = news_df['content'][500]
content_4 = news_df['content'][8882]
content_5 = news_df['content'][8571]
content_6 = news_df['content'][7103]
content_7 = news_df['content'][8098]
content_8 = news_df['content'][3905]
content_9 = news_df['content'][8634]
content_10 = news_df['content'][1000]

print(content_1)
print(content_1_url)
print(content_2)
print(content_3)
print(content_4)
print(content_5)
print(content_6)
print(content_7)
print(content_8)
print(content_9)
print(content_10)



Single Post | themontynews No challenge before us is more important and more potentially life-giving than that we come to see and know our fellow citizens, our neighbors who’ve become strangers. — Krista Tippett The Montgomery News The Voice of Your Community — Serving Belle Mead, Blawenburg, Griggstown, Harlingen, Montgomery, Skillman, and Rocky Hill, NJ Home Advertise Nov 5 General Election Calendar Obituaries More... I'm busy working on my blog posts. Watch this space! Please reload Recent Posts Four Candidates Running for Two Seats in Trenton /... October 31, 2019 Andrew Zwicker (D) Candidate for NJ State Assembly October 31, 2019 Christine Madrid (R) Candidate for NJ Assembly October 31, 2019 Please reload Archive October 2019 September 2019 August 2019 July 2019 June 2019 May 2019 April 2019 March 2019 February 2019 January 2019 December 2018 November 2018 October 2018 September 2018 August 2018 July 2018 June 2018 May 2018 April 2018 March 2018 February 2018 January 2018 Please 

In [45]:
import spacy
 
nlp = spacy.load('en_core_web_lg')
patterns = [{"label": "GPE", "pattern": "Skillman"}]
test_content = news_df['content'][500]
print(test_content)
uc = set()
ruler = nlp.add_pipe("entity_ruler","ruleActions", config={"overwrite_ents": True})
ruler.add_patterns(patterns)

print(f'spaCy Pipelines: {nlp.pipe_names}')

doc = nlp(test_content)
j=0
    
for ent in doc.ents:
    #print(ent, ent.label_)
    if ent.label_ == 'GPE':
        j = j+1
        gpe_text = ent.text
        print("Locations found in content:")
        print(gpe_text)
        uc.add(gpe_text)

print(j)

Single Post | themontynews No challenge before us is more important and more potentially life-giving than that we come to see and know our fellow citizens, our neighbors who’ve become strangers. — Krista Tippett The Montgomery News The Voice of Your Community — Serving Belle Mead, Blawenburg, Griggstown, Harlingen, Montgomery, Skillman, and Rocky Hill, NJ Home News Stories Advertise Calendar Obituaries More... I'm busy working on my blog posts. Watch this space! Please reload Recent Posts 80th Anniversary of Monty’s Fire Department September 30, 2019 Skillman Resident Airlifted to Hospital Following... September 17, 2019 Rocky Hill Clam Rustlers Return September 12, 2019 Please reload Archive September 2019 August 2019 July 2019 June 2019 May 2019 April 2019 March 2019 February 2019 January 2019 December 2018 November 2018 October 2018 September 2018 August 2018 July 2018 June 2018 May 2018 April 2018 March 2018 February 2018 January 2018 Please reload Tags Alfred Hadinger Alice Chen A

In [46]:
import spacy
 
nlp = spacy.load('en_core_web_lg')
patterns = [{"label": "GPE", "pattern": "Skillman"}]
test_content = news_df['content'][500]
test_content = modify_content(test_content)
print(test_content)
uc = set()
ruler = nlp.add_pipe("entity_ruler","ruleActions", config={"overwrite_ents": True})
ruler.add_patterns(patterns)

print(f'spaCy Pipelines: {nlp.pipe_names}')

doc = nlp(test_content)
j=0
    
for ent in doc.ents:
    #print(ent, ent.label_)
    if ent.label_ == 'GPE':
        j = j+1
        gpe_text = ent.text
        print("Locations found in content:")
        print(gpe_text)
        uc.add(gpe_text)

print(j)

Single Post | themontynews No challenge before us is more important and more potentially life-giving than that we come to see and know our fellow citizens, our neighbors who’ve become strangers. — Krista Tippett  Home News Stories Advertise Calendar Obituaries More... I'm busy working on my blog posts. Watch this space! Please reload Recent Posts 80th Anniversary of Monty’s Fire Department September 30, 2019 Skillman Resident Airlifted to Hospital Following... September 17, 2019 Rocky Hill Clam Rustlers Return September 12, 2019 Please reload Archive September 2019 August 2019 July 2019 June 2019 May 2019 April 2019 March 2019 February 2019 January 2019 December 2018 November 2018 October 2018 September 2018 August 2018 July 2018 June 2018 May 2018 April 2018 March 2018 February 2018 January 2018 Please reload Tags Alfred Hadinger Alice Chen Augusta National Barbara Preston Belle Mead Belle Mead NJ Brendan Patrick Welch Carrier Clinic Chase M. Coram Chase M. Coram Obit Declan Kyler
spa